# Hackathon

Some utilities

## Import Utils

In [2]:
import keras
from keras.metrics import top_k_categorical_accuracy
import h5py as h5
import numpy as np
import pandas as pd
import tables

PATH_DATA = '/notebooks/data/full.h5'

In [3]:
def get_idxs(h5_path):
    f = h5.File(h5_path)
    return range(len(f['S2']))

def shuffle_idx(sample_idxs):
    return list(np.random.permutation(sample_idxs))

def split_train_val(sample_idxs, proportion):
    n_samples = len(sample_idxs)
    return sample_idxs[:int((1.-proportion)*n_samples)], sample_idxs[int((1.-proportion)*n_samples):]

def get_batch_count(idxs, batch_size):
    batch_count = int(len(idxs)//batch_size)
    remained_samples = len(idxs)%batch_size
    if remained_samples > 0:
        batch_count += 1

    return batch_count

In [4]:
def generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')
    while True : 
        idxs = shuffle_idx(idxs)
        batch_count = get_batch_count(idxs, batch_size)
        for b in range(batch_count):
            batch_idx = idxs[b*batch_size]
            X = f['S2'][batch_idx:batch_idx+batch_size, :,:,:]
            Y = f['TOP_LANDCOVER'][batch_idx:batch_idx+batch_size, :]
            yield np.array(X), keras.utils.np_utils.to_categorical(np.array(Y), 23)

In [7]:
##DataSet Full
dsetfull = h5.File(PATH_DATA)['TOP_LANDCOVER']
labelsfull = dsetfull[0:].astype(int)
data4Dfull = h5.File(PATH_DATA)['S2']

In [20]:
classes = [1, 2, 3, 4, 5, 10, 11, 12, 14, 19, 20, 17]
equilibre = np.zeros((len(classes), 200000))

for i in range(len(classes)):
        current = classes[i]
        print(current)
        
        if current != 17:
            equilibre[i][:] = np.where(labelsfull == current)[0][:200000]
        else:
             equilibre[i][:86471] = np.where(labelsfull == current)[0][:]

1
2
3
4
5
10
11
12
14
19
20
17


In [19]:
indexs = np.flatten(equilibre)
indexs = indexs[:]
        
indexs = np.sort(indexs)

[]


In [ ]:
##DataSet Equilibré
dset = h5.File(PATH_DATA)['TOP_LANDCOVER'][Equidxs,:]
labels = dset[0:].astype(int)
data4D = h5.File(PATH_DATA)['S2'][Equidxs,:,:,:]

In [ ]:
plt.hist(labels)